In [2]:
# imports
import pandas as pd
import json
import requests # This library will be used to call the APIs
import os # Use this library to access environment variable(s)
from dotenv import load_dotenv # Use this library to load the environment variables into the code
import time # Use this library to pause the program for a specified amount of time

In [27]:
# Load the keys
load_dotenv()
FOURSQUARE_API_KEY = os.getenv('FOURSQUARE_API_KEY')
YELP_API_KEY = os.getenv('YELP_API_KEY')

In [28]:
# Load Citybikes Data
citybikes = pd.read_csv('../data/citybikes.csv')

citybikes.columns

Index(['station_id', 'station_name', 'station_address', 'station_latitude',
       'station_longitude', 'available_bikes', 'empty_bike_slots'],
      dtype='object')

### Testing API connections.

In [29]:
# Test parameters, Lat/Lon for Hamilton
test_lat = 43.2557
test_lon = -79.8711

In [30]:
# Foursquare
def test_foursquare_api():
  url = "https://api.foursquare.com/v3/places/search"
  headers = {
    "Authorization": FOURSQUARE_API_KEY,
    "Accept": "application/json"
  }
  params = {
    "ll": f"{test_lat},{test_lon}",
    "query": "restaurant",
    "radius": 1000,
    "limit": 1  # Keep this minimal for testing
  }
  try:
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    print("Foursquare API Test: SUCCESS")
    print(f"Response Code: {response.status_code}")
  except requests.exceptions.HTTPError as http_err:
    print("Foursquare API Test: FAILED")
    print(f"HTTP Error: {http_err}")
  except Exception as err:
    print("Foursquare API Test: FAILED")
    print(f"Other Error: {err}")
    

# Run tests
print("\nTesting Foursquare API...")
test_foursquare_api()


Testing Foursquare API...
Foursquare API Test: SUCCESS
Response Code: 200


In [9]:
# Yelp
def test_yelp_api():
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": f"Bearer {YELP_API_KEY}",
        "Accept": "application/json"
    }
    params = {
        "latitude": test_lat,
        "longitude": test_lon,
        "categories": "restaurants",
        "radius": 1000,
        "limit": 1
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        print("Yelp API Test: SUCCESS")
        print(f"Response Code: {response.status_code}")
    except requests.exceptions.HTTPError as http_err:
        print("Yelp API Test: FAILED")
        print(f"HTTP Error: {http_err}")
    except Exception as err:
        print("Yelp API Test: FAILED")
        print(f"Other Error: {err}")

# Run tests
print("Testing Yelp API...")
test_yelp_api()

Testing Yelp API...
Yelp API Test: SUCCESS
Response Code: 200


As instructed, we will search for some Points of Interest (POIs). The following have been selected:

- `restaurants` on Yelp -- "13065" on Foursquare
- `museums` on Yelp -- "10027" on Foursquare
-  `amusementparks` on Yelp -- "10001" on Foursquare

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [31]:
# Foursquare
def query_foursquare(lat, lon, categories, radius=1000, limit=50):
      url = "https://api.foursquare.com/v3/places/search"
      headers = {
            'Authorization' : FOURSQUARE_API_KEY,
            'Accept' : 'application/json'
      }
      params = {
      'll': f"{lat},{lon}",
      'categories': categories,
      'radius': radius,
      'limit': limit
      }
  
      try:
            response = requests.get(url, headers=headers, params=params)
            
            # Handle rate limiting
            if response.status_code == 429:
                  retry_after = int(response.headers.get('Retry-After', 20))
                  print(f"Rate limit exceeded. Retrying after {retry_after} seconds.")
                  time.sleep(retry_after)
                  response = requests.get(url, headers=headers, params=params)
                  
            # Raise exception for HTTP errors
            response.raise_for_status()
            data = response.json()
            return data.get('results', [])
      except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err} - Status Code: {response.status_code}")
      except Exception as err:
            print(f"An error occurred: {err}")
    
      return []

In [32]:
# FSQ Categories
categories_fsq = ['13065', # restaurants
                 '10027', # museums
                 '10001'] # amusement parks

In [33]:
# Test Query
query_foursquare(test_lat, test_lon, categories_fsq[0], radius=1000, limit=1)

[{'fsq_id': '55de230f498eecf46f6652a2',
  'categories': [{'id': 13306,
    'name': 'Taco Restaurant',
    'short_name': 'Tacos',
    'plural_name': 'Taco Restaurants',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_',
     'suffix': '.png'}}],
  'chains': [],
  'closed_bucket': 'VeryLikelyOpen',
  'distance': 359,
  'geocodes': {'drop_off': {'latitude': 43.257143, 'longitude': -79.86724},
   'main': {'latitude': 43.257258, 'longitude': -79.867202},
   'roof': {'latitude': 43.257258, 'longitude': -79.867202}},
  'link': '/v3/places/55de230f498eecf46f6652a2',
  'location': {'address': '41 King William St',
   'country': 'CA',
   'cross_street': 'Hughson',
   'formatted_address': '41 King William St (Hughson), Hamilton ON L8R 1A2',
   'locality': 'Hamilton',
   'postcode': 'L8R 1A2',
   'region': 'ON'},
  'name': 'The Mule',
  'related_places': {},
  'timezone': 'America/Toronto'}]

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [34]:
# Store results
fsq_poi_results = []

total_stations = len(citybikes)

In [35]:
# Iterate over each bike station
for index, station in citybikes.iterrows():
    lat = station['station_latitude']
    lon = station['station_longitude']
    station_id = station['station_id']
    station_name = station['station_name'] # Streaming output to console
    print(f"Processing Station {index + 1}/{total_stations}: {station_name}")

    # Iterate over each search term category
    for category in categories_fsq:
        fsq_pois = query_foursquare(lat, lon, category)
        
        # Parse each POI and append to results
        for poi in fsq_pois:
            fsq_poi_data = {
                'fsq_id': poi.get('fsq_id'), 
                'poi_name': poi.get('name'),
                'poi_category_id': category,
                'poi_rating': poi.get('rating'),
                'poi_review_count': poi.get('stats', {}).get('total_ratings'),
                'poi_latitude': poi.get('geocodes', {}).get('main', {}).get('latitude'),
                'poi_longitude': poi.get('geocodes', {}).get('main', {}).get('longitude'),
                'poi_address': poi.get('location', {}).get('formatted_address', []),
                'postal_code': poi.get('location', {}).get('postal_code'),
                'station_id': station_id,
                'distance_from_station': poi.get('distance')
            }
            fsq_poi_results.append(fsq_poi_data)
        print("Entered. Waiting...")
        time.sleep(0.25) # Sleep for 250ms to avoid rate limiting

Processing Station 1/190: Bay at Strachan
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 2/190: MAC Rack ITB
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 3/190: Augusta at John
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 4/190: Hunter GO Centre
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 5/190: Maple at Rothsay - ERI08
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 6/190: Whitney at Lower Horning
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 7/190: Frid at Main
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 8/190: Locke at Hunter
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 9/190: Mac Rack Kingswalk Bridge
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 10/190: James North at Mulberry
Entered.

Put your parsed results into a DataFrame

In [36]:
foursquare = pd.DataFrame(fsq_poi_results)

In [37]:
# Add station name
foursquare['station_name'] = foursquare['station_id'].map(citybikes.set_index('station_id')['station_name'])

print(foursquare.shape)
print(foursquare.columns)

(8698, 12)
Index(['fsq_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code', 'station_id', 'distance_from_station', 'station_name'],
      dtype='object')


In [38]:
foursquare.head(1)

,fsq_id,poi_name,poi_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,station_id,distance_from_station,station_name
0,5ba0f82a15173e002c280c07,Synonym,13065,None,None,43.264558,-79.865376,"328 James St N (Barton Street West), Hamilton ...",None,024a3edf037cb411d16acc08a7fcb954,420,Bay at Strachan


In [39]:
# Convert to CSV
foursquare.to_csv('../data/raw/foursquare.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [10]:
# Yelp
def query_yelp(lat, lon, categories, radius=1000, limit=50):
  url = "https://api.yelp.com/v3/businesses/search"
  headers = {
    "Authorization": f"Bearer {YELP_API_KEY}",
    "Accept": "application/json"
  }
  params = {
    "latitude": lat,
    "longitude": lon,
    "categories": categories,
    "radius": radius,
    "limit": limit
  }
    
  try:
    response = requests.get(url, headers=headers, params=params)
        
    # Handle rate limiting
    if response.status_code == 429:
      retry_after = int(response.headers.get('Retry-After', 20))
      print(f"Rate limit exceeded. Retrying after {retry_after} seconds.")
      time.sleep(retry_after)
      response = requests.get(url, headers=headers, params=params)
    
    # Raise exception for HTTP errors
    response.raise_for_status()
    data = response.json()
    return data.get('businesses', [])
    
  except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err} - Status Code: {response.status_code}")
  except Exception as err:
    print(f"An error occurred: {err}")
    
  return []

In [12]:
# Yelp Categories
categories_yelp = ['restaurants', 'museums', 'amusementparks']

In [50]:
# Test Yelp
query_yelp(test_lat, test_lon, categories_yelp[0], radius=1000, limit=1)

[{'id': '752Fv2jKafftvoS3Twkqyg',
  'alias': 'hambrgr-hamilton',
  'name': 'Hambrgr',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/18TdNrY_d9j0iVlvIcVb3w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/hambrgr-hamilton?adjust_creative=GeuIqjL6ofUJ72HeWCsLHw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GeuIqjL6ofUJ72HeWCsLHw',
  'review_count': 216,
  'categories': [{'alias': 'burgers', 'title': 'Burgers'},
   {'alias': 'beerbar', 'title': 'Beer Bar'}],
  'rating': 4.3,
  'coordinates': {'latitude': 43.25721, 'longitude': -79.8669},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '49 King William Street',
   'address2': None,
   'address3': '',
   'city': 'Hamilton',
   'zip_code': 'L8R 1A2',
   'country': 'CA',
   'state': 'ON',
   'display_address': ['49 King William Street',
    'Hamilton, ON L8R 1A2',
    'Canada']},
  'phone': '+12893891212',
  'display_phone': '+1 289-389-1212',
  'distance': 377.3430524726985,

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [13]:
# Store results
yelp_poi_results = []

total_stations = len(citybikes)
batch = (len(citybikes) / 2)

print(f"total_stations: {total_stations}")
print(f"batch: {batch}")

batch1 = citybikes.iloc[:int(batch)] # First half
batch2 = citybikes.iloc[int(batch):] # Second half

total_stations: 190
batch: 95.0


In [14]:
# Iterate over each bike station
for index, station in batch2.iterrows(): # We are processing by batch.
    lat = station['station_latitude']
    lon = station['station_longitude']
    station_id = station['station_id']
    station_name = station['station_name'] # Streaming output to console
    
    # We are processing by batch.
    print(f"Processing Station {index + 1}/{len(batch2)}: {station_name}")
  
  # Iterate over each search term category group
    for category in categories_yelp:
      yelp_pois = query_yelp(lat, lon, category, radius=1000, limit=50)
    
      # Parse each POI and append to results
      for poi in yelp_pois:
        yelp_poi_data = {
          'yelp_id': poi.get('id'),
          'poi_name': poi.get('name'),
          'poi_category_id': category,
          'poi_rating': poi.get('rating'),
          'poi_review_count': poi.get('review_count'),
          'poi_latitude': poi.get('coordinates', {}).get('latitude'),
          'poi_longitude': poi.get('coordinates', {}).get('longitude'),
          'poi_address': poi.get('location', {}).get('display_address', []),
          'postal_code': poi.get('location', {}).get('zip_code'),
          'station_id': station_id,
          'distance_from_station': poi.get('distance'),
          'station_name': station_name
        }
        
        yelp_poi_results.append(yelp_poi_data)
      print("Entered. Waiting...")
      time.sleep(0.25) # Sleep for 250ms to avoid rate limiting

Processing Station 96/95: MAC Rack BSB
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 97/95: Barton at Ottawa - ERI06
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 98/95: Bay at Macaulay
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 99/95: Emerson at Whitney
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 100/95: MAC Rack ABB East
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 101/95: MAC Rack MDCL Kingswalk
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 102/95: Cannon at Birch (Powell Park)
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 103/95: Macklin at King
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 104/95: AM Cunningham Parkette
Entered. Waiting...
Entered. Waiting...
Entered. Waiting...
Processing Station 105/95: Cannon at 

Put your parsed results into a DataFrame

In [70]:
# Batch 1
yelp1 = pd.DataFrame(yelp_poi_results)

print(yelp1.shape)
print(yelp1.columns)

(4992, 12)
Index(['yelp_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code', 'station_id', 'distance_from_station', 'station_name'],
      dtype='object')


In [71]:
# Backup Batch 1
yelp1.to_csv('../data/raw/yelp1.csv', index=False)

In [15]:
# Batch 2
yelp2 = pd.DataFrame(yelp_poi_results)

print(yelp2.shape)
print(yelp2.columns)

(5030, 12)
Index(['yelp_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code', 'station_id', 'distance_from_station', 'station_name'],
      dtype='object')


In [16]:
# Backup Batch 2
yelp2.to_csv('../data/raw/yelp2.csv', index=False)

In [17]:
# Load Backups
yelp1 = pd.read_csv('../data/raw/yelp1.csv')
yelp2 = pd.read_csv('../data/raw/yelp2.csv')

In [18]:
# Merge
yelp = pd.concat([yelp1, yelp2])

print(yelp.shape)
print(yelp.columns)

(10022, 12)
Index(['yelp_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code', 'station_id', 'distance_from_station', 'station_name'],
      dtype='object')


In [19]:
yelp.head(1)

,yelp_id,poi_name,poi_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,station_id,distance_from_station,station_name
0,fNfZ0bOEvXOxMP1vDEDi8w,Pirate Life Theatre,amusementparks,0.0,0,43.268525,-79.870901,"['200 Harbour Front Drive', 'Hamilton, ON L8L ...",L8L 1C8,024a3edf037cb411d16acc08a7fcb954,242.169774,Bay at Strachan


In [20]:
# Convert to CSV
yelp.to_csv('../data/raw/yelp.csv', index=False)

# Comparing Results

In [41]:
# Load CSVs
foursquare = pd.read_csv('../data/raw/foursquare.csv')
yelp = pd.read_csv('../data/raw/yelp.csv')

In [42]:
# FSQ
print(foursquare.shape)
print(foursquare.nunique())

(8698, 12)
fsq_id                   669
poi_name                 586
poi_category_id            3
poi_rating                 0
poi_review_count           0
poi_latitude             601
poi_longitude            608
poi_address              593
postal_code                0
station_id               190
distance_from_station    986
station_name             190
dtype: int64


In [43]:
# Yelp
print(yelp.shape)
print(yelp.nunique())

(10022, 12)
yelp_id                   529
poi_name                  485
poi_category_id             3
poi_rating                 41
poi_review_count           74
poi_latitude              493
poi_longitude             500
poi_address               494
postal_code               330
station_id                190
distance_from_station    9523
station_name              190
dtype: int64


In [44]:
# Column names, extracting from foursquare -- Doesn't matter, Yelp has the same columns.
a = foursquare.columns
b = a[1:(len(a)-1)]

column_names = [i for i in b]
print(column_names)

['poi_name', 'poi_category_id', 'poi_rating', 'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address', 'postal_code', 'station_id', 'distance_from_station']


In [45]:
# FSQ values
c = foursquare.nunique()
d = c[1:(len(c)-1)]

fsq = [i for i in d]
print(f"Foursquare: {fsq}")

# Yelp values
e = yelp.nunique()
f = e[1:(len(e)-1)]

yelp = [i for i in f]
print(f"Yelp: {yelp}")

Foursquare: [586, 3, 0, 0, 601, 608, 593, 0, 190, 986]
Yelp: [485, 3, 41, 74, 493, 500, 494, 330, 190, 9523]


In [46]:
# Merge
compare = pd.DataFrame(
  {
    'column_names': column_names,
    'foursquare': fsq,
    'yelp': yelp
  }
).reset_index(drop=True)

compare = compare.drop([len(compare)-2]) # Drop station_id row

In [47]:
compare

,column_names,foursquare,yelp
0,poi_name,586,485
1,poi_category_id,3,3
2,poi_rating,0,41
3,poi_review_count,0,74
4,poi_latitude,601,493
5,poi_longitude,608,500
6,poi_address,593,494
7,postal_code,0,330
9,distance_from_station,986,9523


In [49]:
# For showing only
# print(compare.to_markdown(index=False))

## Which API provided you with more complete data? Provide an explanation. 

#### Unique Values -- <i>`(df.nunique())`</i>
| column_names          |   foursquare |   yelp |
|:----------------------|-------------:|-------:|
| poi_name              |          586 |    485 |
| poi_category_id       |            3 |      3 |
| poi_rating            |            0 |     41 |
| poi_review_count      |            0 |     74 |
| poi_latitude          |          601 |    493 |
| poi_longitude         |          608 |    500 |
| poi_address           |          593 |    494 |
| postal_code           |            0 |    330 |
| distance_from_station |          986 |   9523 |

### Initial Observations:
- Foursquare contains more unique POIs (Point of Interest names) than Yelp.
- Yelp provides ratings, while Foursquare does not.
- Yelp provides review counts, while Foursquare does not.
- Foursquare has more unique addresses, suggesting a larger variety of exact POI locations.
- Yelp includes postal codes separately, while Foursquare includes it in the formatted address. I specifically called the complete address from Foursquare. [Here's where we fixed it.](../data/cleaning.ipynb)

### Key Differences:
1. Data Richness
  - Yelp provides more detailed metadata for each POI, including poi_rating and poi_review_count, which are absent or incomplete in Foursquare.
  - Foursquare, however, captures more unique POIs and geographical diversity (poi_latitude, poi_longitude, and poi_address).
2. Distance Data
  - Yelp has significantly more unique distance_from_station values compared to Foursquare (9,523 vs. 986). This could indicate that Yelp uses more precise or dynamic calculations for distances
3. POI Identification
  - Foursquare has more unique poi_name entries (586 vs. 485), suggesting broader POI coverage or naming diversity.

### Considerations for Use:

**I. Coverage**

If the goal is to analyze broader POI coverage with less focus on reviews, Foursquare might be more suitable.

For richer metadata and finer granularity, Yelp is better.

**II. Data Completeness**

Yelp offers ratings and review counts, which can be useful for analyzing popularity or quality.

Foursquare’s broader geographic and POI coverage could be valuable for location-specific analyses.

**III. Integration**

Matching datasets may require careful handling of poi_name and poi_address, as Foursquare has more unique values, potentially with more variations.


Get the top 10 restaurants according to their rating

> The data requires some cleaning. Check out the process [here](../data/cleaning.ipynb).

In [108]:
# FSQ did not come with ratings, so we'll get rating data from Yelp.

# Load cleaned Yelp Data
yelp_cleaned = pd.read_csv('../data/pois_only/pois_yelp.csv')

yelp_cleaned.head()

,yelp_id,poi_name,poi_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code
0,OglGsGS-GdMeSxAvjdzfLQ,2 For 1 Pizza Wings & Subs Restaurant,restaurants,1.5,4,43.245080,-79.807620,158 Kenilworth Avenue N,L8H 4R8
1,XABRVIvmufpfZ2J1llV9MA,28 Lister,restaurants,1.0,1,43.257621,-79.868547,28 James Street L8R,NaN
2,Zr-TN76nzcRVEgJwZHX_7g,5 Alarm Wings And Fish & Chips,restaurants,2.9,8,43.234640,-79.856750,553 Upper Wentworth Street,L9A 4T8
3,CODMFMsPP06eTQutMnniUA,541 Eatery & Exchange,restaurants,4.5,45,43.258248,-79.842354,541 Barton Street E,L8L 2Z2
4,12GctL58qX4G9bS2kBomHg,A & J Sugar Bowl,restaurants,4.3,8,43.260971,-79.868778,124 MacNab Street N,L9H 2K1


In [116]:
sorted = yelp_cleaned.sort_values(by=['poi_rating', 'poi_review_count'], ascending=False)

restaurants = sorted[sorted['poi_category_id'] == 'restaurants']
top_10 = restaurants[['poi_name', 'poi_rating', 'poi_review_count', 'poi_address', 'postal_code']].head(10)

In [118]:
# print(top_10.to_markdown(index=False))

#### Top 10 Restaurants based on their rating.
| poi_name                    |   poi_rating |   poi_review_count | poi_address                | postal_code   |
|:----------------------------|-------------:|-------------------:|:---------------------------|:--------------|
| Tomah                       |            5 |                 12 | 132 Queen Street S         | L8P 3S2       |
| Mystic Ramen                |            5 |                  6 | 51 King William Street     | L8R 1A2       |
| Domino's Pizza              |            5 |                  5 | 86 Main Street Dundas ON   | L9H 2R1       |
| Banh Mi Hue                 |            5 |                  4 | 127 MacNab Street N        | L8R 2B5       |
| Tony G’s Pizzeria           |            5 |                  4 | 30 John Street N           | L8R 1G9       |
| Joey Turks Island Grill     |            5 |                  3 | 178 Locke Street S         | L8P 4B3       |
| Rosales diner               |            5 |                  3 | 303 Dundurn Street S       | L8P 4L4       |
| The Indian Connection Pizza |            5 |                  3 | 1022 Barton Street E       | L8L 3E4       |
| The Red Door Cucina         |            5 |                  3 | 21 King Street W Dundas ON | L9H 1T5       |
| Cozy Sushi                  |            5 |                  2 | 6 Bold Street              | L8P 1T2       |

<br>

> Note: I factored in the Review Count in the sorting. I thought it made more sense if more than 1 person rated the place.